# Step 0.0. Install LightAutoML

In [ ]:
pip install -U lightautoml

# Step 0.1. Import necessary libraries 

In [2]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

ModuleNotFoundError: No module named 'lightautoml'

# Step 0.2. Parameters 

In [1]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 6 * 3600 # Time in seconds for automl run
# TIMEOUT = 40 * 60
TARGET_NAME = 'target'

# Step 0.3. Data load 

In [3]:
%%time

train_data = pd.read_csv('../datasets/tabular-playground-series-may-2021/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

CPU times: user 137 ms, sys: 25.1 ms, total: 162 ms
Wall time: 165 ms


,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,0,1,0,1,0,0,0,0,...,0,0,21,0,0,0,0,0,0,1
1,1,0,0,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,13,2,0,0
3,3,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,1,0,3
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [4]:
test_data = pd.read_csv('../datasets/tabular-playground-series-may-2021/test.csv')
test_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,100000,0,0,0,0,0,0,4,4,0,...,0,0,0,0,0,0,0,0,0,0
1,100001,0,0,1,0,0,0,2,0,7,...,3,0,1,0,0,0,1,0,2,1
2,100002,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,6,0
3,100003,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,6,9,14,3
4,100004,0,0,0,0,0,0,1,0,4,...,1,0,0,0,0,0,0,0,0,0


In [5]:
submission = pd.read_csv('../datasets/tabular-playground-series-may-2021/sample_submission.csv')
submission.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.25,0.25,0.25,0.25
1,100001,0.25,0.25,0.25,0.25
2,100002,0.25,0.25,0.25,0.25
3,100003,0.25,0.25,0.25,0.25
4,100004,0.25,0.25,0.25,0.25


# Step 0.5. Add new features

In [ ]:
def create_gr_feats(data):
    pass
    

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]
print(train_data.shape, test_data.shape)

In [ ]:
train_data.head()

# ========= AutoML preset usage =========


## Step 1. Create Task

In [ ]:
%%time

task = Task('multiclass',)

## Step 2. Setup columns roles

In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

## Step 3. Train on full data 

In [ ]:
%%time 

automl = TabularUtilizedAutoML(task = task, 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               reader_params = {'n_jobs': N_THREADS},
                               configs_list=[
#                                    '../input/lightautoml-configs/conf_0_sel_type_0.yml',
#                                    '../input/lightautoml-configs/conf_1_sel_type_1.yml',
                                   './lightautoml/conf_0_GPU.yml',
                                   './lightautoml/conf_1_GPU.yml'
                               ])
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast', silent = False)
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (20, 10), grid = True)

## Step 4. Predict for test data and check OOF score

In [ ]:
%%time

test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

## Step 5. Prepare submission

In [ ]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('lightautoml_GPU_small_1.csv', index = False)

In [ ]:
submission